In [1]:
import json
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
%load_ext autoreload

%autoreload 2


In [4]:
sys.path.insert(0, os.path.abspath("../optimizer"))
import single_node_profiles as snp
# import single_model_profiles as sm_profs

In [6]:
profs = snp.load_single_node_profiles()

In [ ]:
dag = profiler.get_logical_pipeline("pipeline_three")
with open(os.path.abspath("../results/e2e_profs/systemx/resnet_cascade/slo_500ms/alex_1-r50_1-r152_2-171025_083730.json")) as f:
    sample_run = json.load(f)
scale_factors = profiler.get_node_scale_factors(sample_run, dag.reference_node)
node_configs = profiler.get_node_configs_from_experiment(sample_run)
node_profs = {name : profiler.NodeProfile(name, profs[name]) for name, _ in node_configs.items()}

In [7]:
profs.keys()

dict_keys(['alexnet', 'res152', 'res18', 'res50', 'inception', 'tf-kernel-svm', 'tf-lang-detect', 'tf-log-reg', 'tf-lstm', 'tf-nmt', 'tf-resnet-feats'])

In [ ]:
from IPython.display import display, HTML
display(profs["tf-resnet-feats"])

# aws_profs = res_prof[(res_prof.cloud == "aws")].sort_values("mean_batch_size")
# aws_profs

In [ ]:
profs = sm_profs.load_single_model_profiles()
def plot_smp_thrus(prof, title, plot_gcp=False, plot_aws=False):
    # GCP
    plt.clf()
    fig, (ax_thru, ax_lat) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
    dfs = []
    colors = sns.color_palette(n_colors=5)
    if plot_gcp:
        gcp_profs = prof[(prof.cloud == "gcp")]
        for gpu in ["p100"]:
            for num_cpus in [2]:
                df = gcp_profs[(gcp_profs.gpu_type == gpu)
                             & (gcp_profs.num_cpus_per_replica == num_cpus)]
                df = df.sort_values("mean_batch_size")
                c = colors.pop()
                ax_thru.plot(df["mean_batch_size"], df["mean_throughput_qps"], color=c, label="{gpu}".format(gpu=gpu.upper(), cpu=num_cpus))
                ax_lat.plot(df["mean_batch_size"], df["p99_latency"], color=c, label="{gpu}".format(gpu=gpu.upper(), cpu=num_cpus))

                dfs.append(df)
    if plot_aws:
        aws_profs = prof[(prof.cloud == "aws")]
        for gpu in ["k80", "v100"]:
            for num_cpus in [2]:
                c = colors.pop()
                df = aws_profs[(aws_profs.gpu_type == gpu)
                              & (aws_profs.num_cpus_per_replica == num_cpus)]
                df = df.sort_values("mean_batch_size")
                ax_thru.plot(df["mean_batch_size"], df["mean_throughput_qps"], color=c, label="{gpu}".format(gpu=gpu.upper(), cpu=num_cpus))
                ax_lat.plot(df["mean_batch_size"], df["p99_latency"], color=c, label="{gpu}".format(gpu=gpu.upper(), cpu=num_cpus))

    ax_thru.set_xlabel("Batch size")
    ax_thru.set_ylabel("Throughput (qps)")
    ax_thru.legend(loc=0)
    ax_lat.set_xlabel("Batch size")
    ax_lat.set_ylabel("p99 latency (s)")
    ax_lat.legend(loc=0)
    fig.suptitle(title)
    plt.show()
    return dfs
            
plot_smp_thrus(profs["tf-resnet-feats"], "Resnet 152", plot_gcp=True, plot_aws=True)
# incept_dfs = plot_smp_thrus(profs["inception"], "Inception", plot_gcp=True)

# gcp_results = res_prof[(res_prof.cloud == "gcp") & ()]
# gcp_results

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
for d in incept_dfs:
    display(d)

In [ ]:
def load_results(results_dir):
    fs = os.listdir(results_dir)
    experiments = []
    for exp in fs:
        if exp[-4:] == "json":
            with open(os.path.join(results_dir, exp), "r") as f:
                data = json.load(f)
                experiments.append(data)
        else:
            # print("skipping %s" % os.path.join(results_dir, exp))
            pass
    return experiments

In [ ]:
var_size_results = load_results(
    os.path.abspath("../results/single_model_profs_old/kernel-svm-var-size/"))


mname, df = sm_profs.create_model_profile_df(None, var_size_results)




In [ ]:
df

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
x = np.random.rand(1,4096)
X = np.random.rand(10000,4096)
%timeit k = rbf_kernel(X,x)

In [ ]:
k.dot(x)